In [2]:
import os
import uuid
import base64
import tempfile
import numpy as np
import requests
from pathlib import Path
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import BlipProcessor, BlipForConditionalGeneration
from faster_whisper import WhisperModel
from sentence_transformers import SentenceTransformer
import faiss
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import os
import json
import requests


/Users/devanshk/miniforge3/envs/vllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
import tempfile
import json
from moviepy.video.io.VideoFileClip import VideoFileClip
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

def transcribe_audio_with_whisper(video_path, model_size="base"):
    # Extract audio from video
    video = VideoFileClip(video_path)
    
    # Use a temporary file for the extracted audio
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp_audio:
        audio_path = tmp_audio.name
        video.audio.write_audiofile(audio_path)

    # Transcribe with OpenAI Whisper
    client = OpenAI()
    with open(audio_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )

    os.remove(audio_path)  # Clean up temporary audio file

    segments = transcription.segments
    return segments
def align_transcripts_with_captions(captions, audio_segments):
    aligned = []

    for cap in captions:
        start, end = cap["start_time"], cap["end_time"]
        matched_text = []

        for seg in audio_segments:
            if seg.end >= start and seg.start <= end:
                matched_text.append(seg.text.strip())

        cap["transcript"] = " ".join(matched_text)
        aligned.append(cap)

    return aligned



In [18]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from transformers import BlipProcessor, BlipForConditionalGeneration

def compute_ssim(img1, img2):
    return ssim(img1, img2, channel_axis=-1)

def extract_significant_caption_segments(video_path, fps=1, max_duration=None, ssim_threshold=0.90):
    device = "mps" if torch.mps.is_available() else "cpu"
    
    # Load BLIP
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
    
    cap = cv2.VideoCapture(video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    frame_interval = max(1, int(video_fps / fps))

    frame_idx = 0
    segments = []
    current_caption = None
    current_start = None
    last_frame = None
    timestamp = 0.0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = frame_idx / video_fps
        if max_duration is not None and timestamp > max_duration:
            break

        if frame_idx % frame_interval == 0:
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

            if last_frame is not None:
                similarity = compute_ssim(last_frame, frame_rgb)
                if similarity >= ssim_threshold:
                    frame_idx += 1
                    continue

            last_frame = frame_rgb.copy()

            pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            inputs = processor(images=pil_image, return_tensors="pt").to(device)
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)

            if caption != current_caption:
                if current_caption is not None:
                    segments.append({
                        "caption": current_caption,
                        "start_time": current_start,
                        "end_time": timestamp
                    })
                current_caption = caption
                current_start = timestamp

            print(f"[{timestamp:.2f}s] {caption}")

        frame_idx += 1

    if current_caption:
        segments.append({
            "caption": current_caption,
            "start_time": current_start,
            "end_time": timestamp  # use actual final timestamp here
        })

    cap.release()
    return segments


In [63]:
# Example usage
video_path = "video2.mp4"
captions = extract_significant_caption_segments(video_path, fps=60)

# Step 2: Transcribe with Whisper
segments = transcribe_audio_with_whisper("video2.mp4", model_size="base")

# Step 3: Align and merge
final_output = align_transcripts_with_captions(captions, segments)

# Step 4: Save to JSON
with open("captioned_segments_with_audio.json", "w") as f:
    json.dump(final_output, f, indent=2)

[0.00s] a black background with a white and red flower
[0.07s] a view of a cloudy sky at dusk
[0.33s] a view of a cloudy sky at dusk
[0.63s] a view of a cloudy sky at dusk
[0.97s] a very cloudy sky
[1.33s] a cloudy sky
[1.77s] a very cloudy sky
[2.27s] a very cloudy sky
[2.67s] a man and a boy are standing in front of trees
[2.70s] a man and a boy are standing in front of trees
[2.73s] a man and a woman standing in front of trees
[2.80s] a man and a woman standing in front of trees
[2.90s] a man and a boy are standing in front of trees
[2.93s] a man and a boy are standing in front of trees
[2.97s] a man and a boy are standing in front of trees
[3.00s] a man and a boy are standing in front of trees
[3.03s] a man and a boy are standing in front of trees
[3.13s] a man and a boy are standing in front of trees
[3.20s] a man and a boy are standing in front of trees
[3.30s] a man and a boy are standing in front of trees
[3.37s] a man and a boy are standing in front of trees
[3.43s] a man and 

MoviePy - Done.


TypeError: 'TranscriptionSegment' object is not subscriptable

In [64]:
print(captions)

[{'caption': 'a black background with a white and red flower', 'start_time': 0.0, 'end_time': 0.06666666666666667}, {'caption': 'a view of a cloudy sky at dusk', 'start_time': 0.06666666666666667, 'end_time': 0.9666666666666667}, {'caption': 'a very cloudy sky', 'start_time': 0.9666666666666667, 'end_time': 1.3333333333333333}, {'caption': 'a cloudy sky', 'start_time': 1.3333333333333333, 'end_time': 1.7666666666666666}, {'caption': 'a very cloudy sky', 'start_time': 1.7666666666666666, 'end_time': 2.6666666666666665}, {'caption': 'a man and a boy are standing in front of trees', 'start_time': 2.6666666666666665, 'end_time': 2.7333333333333334}, {'caption': 'a man and a woman standing in front of trees', 'start_time': 2.7333333333333334, 'end_time': 2.9}, {'caption': 'a man and a boy are standing in front of trees', 'start_time': 2.9, 'end_time': 5.0}, {'caption': 'a man and a woman are standing in front of trees', 'start_time': 5.0, 'end_time': 5.033333333333333}, {'caption': 'a man a

In [65]:
print(segments)

[TranscriptionSegment(id=0, avg_logprob=-0.22369791567325592, compression_ratio=1.6864111423492432, end=5.320000171661377, no_speech_prob=0.0010810723761096597, seek=0, start=0.0, temperature=0.0, text=' When I think of all of the times in my life that I have felt truly alive, I realize that', tokens=[50364, 1133, 286, 519, 295, 439, 295, 264, 1413, 294, 452, 993, 300, 286, 362, 2762, 4908, 5465, 11, 286, 4325, 300, 50630]), TranscriptionSegment(id=1, avg_logprob=-0.22369791567325592, compression_ratio=1.6864111423492432, end=10.399999618530273, no_speech_prob=0.0010810723761096597, seek=0, start=5.320000171661377, temperature=0.0, text=' all have been a result of me taking a leap of faith and putting myself in an uncomfortable', tokens=[50630, 439, 362, 668, 257, 1874, 295, 385, 1940, 257, 19438, 295, 4522, 293, 3372, 2059, 294, 364, 10532, 50884]), TranscriptionSegment(id=2, avg_logprob=-0.22369791567325592, compression_ratio=1.6864111423492432, end=11.399999618530273, no_speech_prob

In [67]:
final_output = align_transcripts_with_captions(captions, segments)

In [69]:
# Save the final_output to a JSON file
with open("final_output.json", "w") as json_file:
    json.dump(final_output, json_file, indent=2)

In [76]:
from sentence_transformers import SentenceTransformer, util

def smooth_segments(segments, threshold=0.9):
    """
    Merge adjacent segments with semantically similar captions.
    """
    # 1. Load ST model and precompute embeddings for all captions
    st_model = SentenceTransformer("all-MiniLM-L6-v2")
    captions = [seg["caption"] for seg in segments]
    # Compute embeddings in one batch
    embeds = st_model.encode(captions, convert_to_tensor=True, show_progress_bar=True)

    smoothed = []
    i = 0
    n = len(segments)

    while i < n:
        # Start of a new run
        j = i
        # Grow run while next caption is similar enough
        while j + 1 < n and util.cos_sim(embeds[i], embeds[j+1]) >= threshold:
            j += 1
        # Build merged segment from i..j
        merged_caption = segments[i]["caption"]
        start_time = segments[i]["start_time"]
        end_time = segments[j]["end_time"]
        # Keep only the first transcript to avoid repetition
        merged_transcript = segments[i]["transcript"].strip()

        smoothed.append({
            "caption": merged_caption,
            "start_time": start_time,
            "end_time": end_time,
            "transcript": merged_transcript
        })
        # Move to next group
        i = j + 1

    return smoothed


In [77]:
raw = final_output                  # your list of {caption,start_time,end_time,transcript}
smoothed = smooth_segments(raw, threshold=0.95)

Batches: 100%|██████████| 26/26 [00:01<00:00, 20.62it/s]


In [78]:
with open("smoothed_output.json", "w") as json_file:
    json.dump(smoothed, json_file, indent=2)

## JUST EMBEDDING THE CAPTIONS

In [79]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Load smoothed data
with open("smoothed_output.json", "r") as f:
    data = json.load(f)

captions = [item["caption"] for item in data]

# Embed captions
model = SentenceTransformer("all-MiniLM-L6-v2")  # swap out with larger if needed
embeddings = model.encode(captions, convert_to_numpy=True, normalize_embeddings=True)

# Build FAISS index (cosine similarity → IndexFlatIP)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

# Save index and metadata
faiss.write_index(index, "video_captions.index")
with open("video_metadata.json", "w") as f:
    json.dump(data, f)

In [80]:
def retrieve_relevant_segments(query, top_k=5):
    # Load model, index, and metadata
    model = SentenceTransformer("all-MiniLM-L6-v2")
    index = faiss.read_index("video_captions.index")
    with open("video_metadata.json", "r") as f:
        metadata = json.load(f)

    # Encode query
    query_vec = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    # Search
    scores, indices = index.search(query_vec, top_k)

    # Get results
    results = []
    for i, score in zip(indices[0], scores[0]):
        segment = metadata[i]
        results.append({
            "caption": segment["caption"],
            "start_time": segment["start_time"],
            "end_time": segment["end_time"],
            "transcript": segment["transcript"],
            "score": float(score)
        })
    return results


In [81]:
results = retrieve_relevant_segments("a boy smiling under the tree", top_k=3)
for r in results:
    print(f"[{r['start_time']} - {r['end_time']}]: {r['caption']}")
    print(f"Transcript: {r['transcript']}")
    print(f"Score: {r['score']:.4f}\n")


[82.43333333333334 - 82.66666666666667]: a young boy in a forest with trees
Transcript: I'm not gonna lie, lately I have been kind of lost in life, knowing what to do but not
Score: 0.6404

[82.33333333333333 - 82.36666666666666]: a young boy in a forest with trees
Transcript: I'm not gonna lie, lately I have been kind of lost in life, knowing what to do but not
Score: 0.6404

[82.26666666666667 - 82.3]: a young boy in a forest with trees
Transcript: I'm not gonna lie, lately I have been kind of lost in life, knowing what to do but not
Score: 0.6404



## Embedding the Transcription + captions

In [82]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Load smoothed data
with open("smoothed_output.json", "r") as f:
    data = json.load(f)

# Combine caption + transcript
combined_texts = [f"{item['caption']} {item['transcript']}" for item in data]

# Sentence transformer for embedding
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(combined_texts, convert_to_numpy=True, normalize_embeddings=True)

# Create FAISS index (cosine similarity)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

# Save index and metadata
faiss.write_index(index, "video_combined.index")
with open("video_metadata.json", "w") as f:
    json.dump(data, f)


In [83]:
def search_video_segments(query, top_k=5):
    # Load
    model = SentenceTransformer("all-MiniLM-L6-v2")
    index = faiss.read_index("video_combined.index")
    with open("video_metadata.json", "r") as f:
        metadata = json.load(f)

    # Embed query
    query_vec = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    # Search FAISS
    scores, indices = index.search(query_vec, top_k)

    # Format results
    results = []
    for i, score in zip(indices[0], scores[0]):
        item = metadata[i]
        results.append({
            "start_time": item["start_time"],
            "end_time": item["end_time"],
            "caption": item["caption"],
            "transcript": item["transcript"],
            "score": float(score)
        })
    return results


In [84]:
query = "What did the man and boy do?"
results = search_video_segments(query)

for r in results:
    print(f"[{r['start_time']} - {r['end_time']}s]")
    print(f"Caption: {r['caption']}")
    print(f"Transcript: {r['transcript']}")
    print(f"Score: {r['score']:.4f}\n")


[5.7 - 5.8s]
Caption: a man and a boy are standing together
Transcript: all have been a result of me taking a leap of faith and putting myself in an uncomfortable
Score: 0.4326

[5.6 - 5.633333333333334s]
Caption: a man and a boy are laughing together
Transcript: all have been a result of me taking a leap of faith and putting myself in an uncomfortable
Score: 0.4158

[5.5 - 5.566666666666666s]
Caption: a man and a boy are laughing together
Transcript: all have been a result of me taking a leap of faith and putting myself in an uncomfortable
Score: 0.4158

[5.2 - 5.3s]
Caption: a man and a boy are smiling and standing together
Transcript: When I think of all of the times in my life that I have felt truly alive, I realize that
Score: 0.3969

[5.633333333333334 - 5.7s]
Caption: a man and a boy are standing in front of trees
Transcript: all have been a result of me taking a leap of faith and putting myself in an uncomfortable
Score: 0.3702



In [91]:
from openai import OpenAI

client = OpenAI()

def answer_question_with_gpt(query):
    context_segments = search_video_segments(query, top_k=5)
    print(f"Context Segments: {context_segments}")
    context_str = "\n\n".join([
        f"[{seg['start_time']} - {seg['end_time']}s]\nCaption: {seg['caption']}\nTranscript: {seg['transcript']}"
        for seg in context_segments
    ])
    print(f"Context String: {context_str}")
    system_prompt = "You are an assistant answering questions about a video using provided captions and transcripts."

    user_prompt = f"""Context:
{context_str}

Question: {query}
Answer:"""

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    print(f"Completion: {completion}")
    response = completion.choices[0].message.content

    return response


In [92]:
def format_context(segments):
    context = ""
    for seg in segments:
        context += f"[{seg['start_time']} - {seg['end_time']}s] Caption: {seg['caption']}\nTranscript: {seg['transcript']}\n\n"
    return context


In [93]:
query = "What did the man and boy do?"
answer = answer_question_with_gpt(query)
print(answer)

Context Segments: [{'start_time': 5.7, 'end_time': 5.8, 'caption': 'a man and a boy are standing together', 'transcript': 'all have been a result of me taking a leap of faith and putting myself in an uncomfortable', 'score': 0.4325793385505676}, {'start_time': 5.6, 'end_time': 5.633333333333334, 'caption': 'a man and a boy are laughing together', 'transcript': 'all have been a result of me taking a leap of faith and putting myself in an uncomfortable', 'score': 0.41583603620529175}, {'start_time': 5.5, 'end_time': 5.566666666666666, 'caption': 'a man and a boy are laughing together', 'transcript': 'all have been a result of me taking a leap of faith and putting myself in an uncomfortable', 'score': 0.41583603620529175}, {'start_time': 5.2, 'end_time': 5.3, 'caption': 'a man and a boy are smiling and standing together', 'transcript': 'When I think of all of the times in my life that I have felt truly alive, I realize that', 'score': 0.3969244062900543}, {'start_time': 5.633333333333334,